# Fiddler LLM Application Quick Start Guide

Fiddler is the pioneer in enterprise AI Observability, offering a unified platform that enables all model stakeholders to monitor model performance and to investigate the true source of model degredation.  Fiddler's AI Observability platform supports both traditional ML models as well as Generative AI applications.  This guide walks you through how to onboard a LLM chatbot application that is built using a RAG architecture.

---

You can start using Fiddler ***in minutes*** by following these 8 quick steps:

1. Imports
2. Connect to Fiddler
3. Create a Fiddler project
4. Load a data sample
5. Opt-in to specific Fiddler's LLM Enrichments
6. Add information about the LLM application
7. Publish production events
8. Get insights

**Don't have a Fiddler account? [Sign-up for a 14-day free trial](https://www.fiddler.ai/trial?utm_source=fiddler_docs&utm_medium=referral).**

## 1. Imports

In [17]:
import numpy as np
import pandas as pd
import time as time
import fiddler as fdl
import datetime

print(f"Running client version {fdl.__version__}")

Running client version 3.2.0


## 2. Connect to Fiddler

Before you can add information about your LLM application with Fiddler, you'll need to connect using our API client.


---


**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your authorization token

In [ ]:
URL = ''  # Make sure to include the full URL (including https://).
TOKEN = ''

These parameters can be found on the **Settings** page of your Fiddler environment.

Now just run the following code block to connect to the Fiddler API!

In [ ]:
fdl.init(
    url=URL,
    token=TOKEN
)

## 3. Create a Fiddler Project

Once you connect, you can create a new project using the client's `Project` class.

In [ ]:
PROJECT_NAME = 'fiddler_chatbot_test'

In [ ]:
project = fdl.Project(
    name=PROJECT_NAME
)

project.create()

## 4. Load a data sample

In this example, we'll be onboarding data in order to observe our **Fiddler chatbot application**.
  
In order to get insights into the model's performance, **Fiddler needs a small sample of data** to learn the schema of incoming data.
Let's use a file with some historical prompts, source docs, and responses from our chatbot for our sample.

In [ ]:
PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/chatbot_data_sample.csv'

sample_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
sample_df

Fiddler will uses this data sample to keep track of important information about your data.  This includes **data types**, **data ranges**, and **unique values** for categorical variables.

## 5. Opt-in to specific Fiddler LLM Enrichments

After picking a sample of our chatbot's prompts and responses, we can request that Fiddler execute a series of enrichment services that can "score" our prompts and responses for a variety of insights.  These enrichment services can detect AI safety issues like PII leakage, hallucinations, toxicity, and more.  We can also opt-in for enrichment services like embedding generation which will allow us to track prompt and response outliers and drift. A full description of these enrichments can be found [here](https://docs.fiddler.ai/docs/enrichments-beta).

---

Let's define the enrichment services we'd like to use.  Here we will opt in for embedding generation for our prompts, responses and source docs.  Additionally, let's opt in for PII detection, outlier detection through centroid distance metrics, and some other text based evaluation scores.

In [12]:
fiddler_backend_enrichments = [

    #prompt enrichemnts
    fdl.Enrichment(
        name='Enrichment Prompt Embedding',
        enrichment='embedding',
        columns=['question'],
    ),
    fdl.TextEmbedding(
        name='Prompt TextEmbedding',
        source_column='question',
        column='Enrichment Prompt Embedding',
        n_tags=10
    ),
    #response enrichments
     fdl.Enrichment(
        name='Enrichment Response Embedding',
        enrichment='embedding',
        columns=['response'],
    ),
    fdl.TextEmbedding(
        name='Response TextEmbedding',
        source_column='response',
        column='Enrichment Response Embedding',
        n_tags=10
    ),
    #rag document enrichments
    fdl.TextEmbedding(
        name='Source Docs TextEmbedding',
        source_column='source_docs',
        column='Enrichment Source Docs Embedding',
        n_tags=10
    ),
    fdl.Enrichment(
        name='Enrichment Source Docs Embedding',
        enrichment='embedding',
        columns=['source_docs'],
    ),    
    #safety
    fdl.Enrichment(
        name='FTL Safety',
        enrichment='ftl_prompt_safety',
        columns=['question', 'response'],
    ),
    #text quality
    fdl.Enrichment(
        name='Enrichment QA TextStat',
        enrichment='textstat',
        columns=['question', 'response'],
        config={'statistics': [
                'char_count',
                'flesch_reading_ease',
                'flesch_kincaid_grade',
            ]
        }
    ),
    fdl.Enrichment(
        name='Enrichment QA Sentiment',
        enrichment='sentiment',
        columns=['question', 'response'],
    )
]

## 6.  Add information about the LLM application

Now it's time to onboard information about our LLM application to Fiddler.  We do this by defining a `ModelSpec` object.


---


The `ModelSpec` object will contain some **information about how your LLM application operates**.
  
*Just include:*
1. The **input/output** columns.  For a LLM application, these are just the raw inputs and outputs of our LLM application.
2. Any **metadata** columns.
3. The **custom features** which contain the configuration of the enrichments we opted for.

We'll also want a few other pieces of information:
1. The **task** your model is performing (LLM, regression, binary classification, etc.)
2. Which column to use to read timestamps from.

In [13]:
model_spec = fdl.ModelSpec(
    inputs=['question', 'response', 'source_docs'],
    metadata=['session_id', 'comment', 'timestamp', 'feedback'],
    custom_features=fiddler_backend_enrichments
)

model_task = fdl.ModelTask.LLM

timestamp_column = 'timestamp'

Then just publish all of this to Fiddler by creating a `Model` object.

In [14]:
MODEL_NAME = 'rag_chatbot'

model = fdl.Model.from_data(
    source=sample_df,
    name=MODEL_NAME,
    project_id=project.id,
    spec=model_spec,
    task=model_task,
    event_ts_col=timestamp_column
)

model.create()

## 7. Publish production events

Information about our LLM application is onboarded to Fiddler and now it's time to start publishing some production data!  
Fiddler will **monitor this data and compare it to your baseline to generate powerful insights into how your model is behaving**.


---


Each record sent to Fiddler is called **an event**.  Events simply contain the inputs and outputs of a predictive model or LLM application.
  
Let's load in some sample events (prompts and responses) from a CSV file.

In [15]:
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/chatbot_production_data.csv'

llm_events_df = pd.read_csv(PATH_TO_EVENTS_CSV)

# Timeshifting the timestamp column in the events file so the events are as recent as today
llm_events_df['timestamp'] = pd.to_datetime(llm_events_df['timestamp'])
time_diff = pd.Timestamp.now().normalize() - llm_events_df['timestamp'].max()
llm_events_df['timestamp'] += time_diff

llm_events_df

,session_id,source_docs,response,question,comment,feedback,timestamp
0,6d9917af-c889-40a5-bd83-a7860318a26f,"Document: slug: ""cv-monitoring"" _ID, ''moni...","Yes, Fiddler supports LLM (Lifecycle Model Mon...",Does Fiddler support LLM monitoring?,NaN,NaN,2024-02-24 02:29:18.094
1,6d9917af-c889-40a5-bd83-a7860318a26f,"Document: ---\ntitle: ""Designing a Baseline...",A baseline dataset is a representative sample ...,what is baseline dataset?,NaN,Like,2024-02-24 02:30:48.656
2,6d9917af-c889-40a5-bd83-a7860318a26f,"Document: ---\ntitle: ""Baselines""\nslug: ""f...","No, the default baseline for all monitoring me...",Does Fiddler have the ability to change the mo...,NaN,Like,2024-02-24 02:33:35.860
3,6d9917af-c889-40a5-bd83-a7860318a26f,"Document: ---\ntitle: ""Data Drift""\nslug: ""...",The calculation of data drift metrics in Fiddl...,how is data drift metrics calculated?,pretty good,Like,2024-02-24 02:35:27.871
4,6d9917af-c889-40a5-bd83-a7860318a26f,"Document: ---\ntitle: ""Deploying Fiddler""\n...","Yes, it is possible to deploy Fiddler on data ...",Can Fiddler be deployed on data-centers?,nice!,Like,2024-02-24 02:38:26.411
...,...,...,...,...,...,...,...
441,1b88a642-1d2c-4560-821e-ad3ea338ec99,Document: ResourceLink:https://www.fiddler....,Krishna Gade is the Founder and CEO of Fiddler...,Who is Krisha GaDE?,NaN,NaN,2024-06-24 02:38:46.767
442,e71adb1a-c425-4e46-ad61-ec731bdeadb4,Document: ResourceLink:https://www.fiddler....,"Yes, Fiddler can observe LLM applications. Fid...",Can Fiddler Observe LLM APplicaitons?,NaN,NaN,2024-06-24 20:52:41.327
443,76499c4d-77c5-471c-b0aa-2584c535a28d,Document: BlogLink:https://www.fiddler.ai/b...,"Yes, Fiddler supports LLM (Language Model Moni...",Can Fiddler Support LLM Observability?,NaN,NaN,2024-06-24 21:07:49.141
444,78845331-55ca-45f1-8624-f4910025bec9,Document: ResourceLink:https://www.fiddler....,Fiddler supports LLMs by providing tools and r...,How does Fiddler support LLMs?,NaN,Like,2024-06-24 23:17:21.129


You can use the model's `publish` function to start pumping data into Fiddler!
  
Just pass in the DataFrame containing your events.

In [16]:
output = model.publish(llm_events_df)

# 8. Get insights

**You're all done!**
  
You can now head to your Fiddler environment and start getting enhanced observability into your LLM application's performance.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/LLM_chatbot_UMAP.png" />
        </td>
    </tr>
</table>

**What's Next?**

Try the [NLP Monitoring - Quickstart Notebook](https://docs.fiddler.ai/docs/simple-nlp-monitoring-quick-start)

---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

Join our [community Slack](http://fiddler-community.slack.com/) to ask any questions!

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.